In [2]:
import numpy as np
import pandas as pd
import math
import re
from collections import Counter
from functools import cmp_to_key
from itertools import product, permutations, combinations, combinations_with_replacement
from itertools import repeat

In [1]:
with open("13-input", "r") as file:
    lines = file.readlines()
data_raw = [line.replace("\n", "") for line in lines]
data_raw = "\n".join(data_raw)
data_raw

'...##....\n.#.##.#..\n.#....#..\n.##..##..\n...##....\n..####.##\n#..##..##\n\n##...##..\n.##.#..#.\n.#.......\n...#.##.#\n##..#..#.\n.##..##..\n.#...##..\n#.#######\n....####.\n..##.##.#\n..##.##.#\n....####.\n#.#######\n.#...##..\n.##..##..\n#...#..#.\n...#.##.#\n\n...#...##...###\n##.#.##.#..#.##\n####.##.#..#.##\n#.#.##.###.#.##\n#...##.##...#.#\n##..#.###...#..\n#...####.###...\n#.#####...##.##\n#.#####...##.##\n#...####.###...\n##..#.###...#..\n#...##.##...#.#\n#.#.##.###.#.##\n####.##.#..#.##\n##.#.##.#..#.##\n\n.....#.\n###.###\n##.#.##\n..#..##\n..###..\n..###..\n..#..##\n##.#.##\n###.###\n.....##\n###..#.\n..#..##\n##.####\n\n....#....##....\n.##..##.####.##\n#..#.#...##...#\n.##.####....###\n#..#.##.####.##\n#..###.#.##.#.#\n.##.##.##..####\n\n..##..##..#.#.#\n.####......#.#.\n.#.#.###...#..#\n.##.#..##.....#\n..#.#.##..###.#\n..#.#.##..###.#\n.##.#..##.....#\n.#.#.###...#..#\n.####......#.#.\n..##..##..#.#.#\n#...#..#...#...\n.#.#.##..##...#\n.#.#.##..###..#\n\n#...##...#.

In [4]:
test_data_raw = """#.##..##.
..#.##.#.
##......#
##......#
..#.##.#.
..##..##.
#.#.##.#.

#...##..#
#....#..#
..##..###
#####.##.
#####.##.
..##..###
#....#..#"""



def preprocess_data (data):
    patterns = data.split("\n\n")
    patterns_parsed = []
    for pattern in patterns:
        patterns_parsed.append(np.array([list(row)  for row in pattern.split("\n")], dtype='U10'))
    
    return patterns_parsed

test_data = preprocess_data(test_data_raw)
test_data

[array([['#', '.', '#', '#', '.', '.', '#', '#', '.'],
        ['.', '.', '#', '.', '#', '#', '.', '#', '.'],
        ['#', '#', '.', '.', '.', '.', '.', '.', '#'],
        ['#', '#', '.', '.', '.', '.', '.', '.', '#'],
        ['.', '.', '#', '.', '#', '#', '.', '#', '.'],
        ['.', '.', '#', '#', '.', '.', '#', '#', '.'],
        ['#', '.', '#', '.', '#', '#', '.', '#', '.']], dtype='<U10'),
 array([['#', '.', '.', '.', '#', '#', '.', '.', '#'],
        ['#', '.', '.', '.', '.', '#', '.', '.', '#'],
        ['.', '.', '#', '#', '.', '.', '#', '#', '#'],
        ['#', '#', '#', '#', '#', '.', '#', '#', '.'],
        ['#', '#', '#', '#', '#', '.', '#', '#', '.'],
        ['.', '.', '#', '#', '.', '.', '#', '#', '#'],
        ['#', '.', '.', '.', '.', '#', '.', '.', '#']], dtype='<U10')]

In [5]:
data = preprocess_data(data_raw)
data

[array([['.', '.', '.', '#', '#', '.', '.', '.', '.'],
        ['.', '#', '.', '#', '#', '.', '#', '.', '.'],
        ['.', '#', '.', '.', '.', '.', '#', '.', '.'],
        ['.', '#', '#', '.', '.', '#', '#', '.', '.'],
        ['.', '.', '.', '#', '#', '.', '.', '.', '.'],
        ['.', '.', '#', '#', '#', '#', '.', '#', '#'],
        ['#', '.', '.', '#', '#', '.', '.', '#', '#']], dtype='<U10'),
 array([['#', '#', '.', '.', '.', '#', '#', '.', '.'],
        ['.', '#', '#', '.', '#', '.', '.', '#', '.'],
        ['.', '#', '.', '.', '.', '.', '.', '.', '.'],
        ['.', '.', '.', '#', '.', '#', '#', '.', '#'],
        ['#', '#', '.', '.', '#', '.', '.', '#', '.'],
        ['.', '#', '#', '.', '.', '#', '#', '.', '.'],
        ['.', '#', '.', '.', '.', '#', '#', '.', '.'],
        ['#', '.', '#', '#', '#', '#', '#', '#', '#'],
        ['.', '.', '.', '.', '#', '#', '#', '#', '.'],
        ['.', '.', '#', '#', '.', '#', '#', '.', '#'],
        ['.', '.', '#', '#', '.', '#', '#', '.', 

In [21]:
pattern = test_data[1]

def search_mirror_by_cols (pattern):
    size = pattern.shape[1]
    for start_col in range(size-1):
        num_comparisons = min(start_col+1, size-start_col-1)
        mirror_found = True
        print("starting on", start_col)
        for comp_col in range(num_comparisons):
            a = "".join(pattern[:,start_col-comp_col].tolist())
            b = "".join(pattern[:,start_col+comp_col+1].tolist())
            if a != b:
                mirror_found = False
                break

        if mirror_found:
            return start_col
    return -1

print(search_mirror_by_cols(pattern))
print(search_mirror_by_cols(pattern.T))
        # print("comparing", start_col-comp_col, "with", start_col+comp_col+1)

starting on 0
starting on 1
starting on 2
starting on 3
starting on 4
starting on 5
starting on 6
starting on 7
-1
starting on 0
starting on 1
starting on 2
starting on 3
3


In [26]:
def search_mirror_by_cols (pattern):
    size = pattern.shape[1]
    for start_col in range(size-1):
        num_comparisons = min(start_col+1, size-start_col-1)
        mirror_found = True
        # print("starting on", start_col)
        for comp_col in range(num_comparisons):
            a = "".join(pattern[:,start_col-comp_col].tolist())
            b = "".join(pattern[:,start_col+comp_col+1].tolist())
            if a != b:
                mirror_found = False
                break

        if mirror_found:
            return start_col
    return -1   


def solution (data, verbose=False):
    sum_of_mirrors = 0
    for pattern in data:
        cols_mirror = search_mirror_by_cols (pattern)
        rows_mirror = search_mirror_by_cols (pattern.T)

        if cols_mirror != -1:
            sum_of_mirrors += cols_mirror+1

        if rows_mirror != -1:
            sum_of_mirrors += (rows_mirror+1)*100

        if (cols_mirror != -1) and (rows_mirror != -1):
            print("Found mirror in cols and rows")


    return sum_of_mirrors


sol = solution(data)
# sol = solution(test_data, verbose=True)

print(sol)
# display(sum(sol))


30518


# Part 2

In [29]:
def search_mirror_by_cols (pattern):
    size = pattern.shape[1]
    for start_col in range(size-1):
        num_comparisons = min(start_col+1, size-start_col-1)
        mirror_found = True
        one_error = False
        # print("starting on", start_col)
        for comp_col in range(num_comparisons):
            a = "".join(pattern[:,start_col-comp_col].tolist())
            b = "".join(pattern[:,start_col+comp_col+1].tolist())
            if one_error:
                if a != b:
                    mirror_found = False
                    break
            else:
                if a != b: 
                    errors = sum(a != b for a, b in zip(a, b))
                    if errors == 1:
                        one_error = True
                    else:
                        mirror_found = False
                        break

        if mirror_found and one_error:
            return start_col
    return -1   


def solution (data, verbose=False):
    sum_of_mirrors = 0
    for pattern in data:
        cols_mirror = search_mirror_by_cols (pattern)
        rows_mirror = search_mirror_by_cols (pattern.T)

        if cols_mirror != -1:
            sum_of_mirrors += cols_mirror+1

        if rows_mirror != -1:
            sum_of_mirrors += (rows_mirror+1)*100

        if (cols_mirror != -1) and (rows_mirror != -1):
            print("Found mirror in cols and rows")


    return sum_of_mirrors


sol = solution(data)
# sol = solution(test_data, verbose=True)

print(sol)
# display(sum(sol))


36735
